In [1]:
!pip install emoji

In [2]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import string
import emoji

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer



from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import classification_report

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nithin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nithin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nithin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Load your CSV file into a pandas DataFrame
csv_file_path = 'DATSETS/sample_text - Sheet1.csv'  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path)

In [11]:
df

,tweet_id,text,task1
0,1123757263427186690,"hate wen females hit ah nigga with tht bro 😂😂,...",HOF
1,1123733301397733380,RT @airjunebug: When you're from the Bay but y...,HOF
2,1123734094108659712,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NOT
3,1126951188170199049,RT @SheLoveTimothy: He ain’t on drugs he just ...,HOF
4,1126863510447710208,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NOT
...,...,...,...
995,1126798721025544193,"RT @prodnose: Good morning, everyone.\nFollowi...",NOT
996,1126833089190219777,@cheezitking123 this what you get for tryna ge...,NOT
997,1130037092845670400,earphones ko 😭😭😭😭😭😭😭,NOT
998,1127028455651123201,RT @nj_linguist: @realgonegirl @elivalley I th...,NOT


PREPROCESSING

In [12]:
ps = PorterStemmer()
lemmatiser = WordNetLemmatizer()
english_stopwords = set(stopwords.words('english'))
exclude = set(string.punctuation)

def preprocess(text):
    # Convert to lowercase
    text = text.lower()

    # Remove digits
    text = re.sub(r'\d+', '', text)

    # Remove HTML tags
    text = re.sub('<.*?>', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Remove punctuation
    text = ''.join(ch for ch in text if ch not in exclude)

    # Tokenize, lemmatize, and stem
    tokens = word_tokenize(text)
    tokens = [lemmatiser.lemmatize(t) for t in tokens]
    tokens = [ps.stem(t) for t in tokens]

    # Remove stopwords
    tokens = [t for t in tokens if t not in english_stopwords]

    # Join tokens back into text
    text = " ".join(tokens)

    # Demojize and replace underscores
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = text.replace("_", " ")

    return text


In [13]:
df['processed_text'] = df['text'].astype(str).apply(preprocess)

In [14]:
df

,tweet_id,text,task1,processed_text
0,1123757263427186690,"hate wen females hit ah nigga with tht bro 😂😂,...",HOF,hate wen femal hit ah nigga tht bro im tryna m...
1,1123733301397733380,RT @airjunebug: When you're from the Bay but y...,HOF,rt airjunebug bay realli ny nigga heart w supp...
2,1123734094108659712,RT @DonaldJTrumpJr: Dear Democrats: The Americ...,NOT,rt donaldjtrumpjr dear democrat american peopl...
3,1126951188170199049,RT @SheLoveTimothy: He ain’t on drugs he just ...,HOF,rt shelovetimothi aint drug bore shit im bore
4,1126863510447710208,RT @TavianJordan: Summer ‘19 I’m coming for yo...,NOT,rt tavianjordan summer im come bore shit beach...
...,...,...,...,...
995,1126798721025544193,"RT @prodnose: Good morning, everyone.\nFollowi...",NOT,rt prodnos good morn everyon follow one worst ...
996,1126833089190219777,@cheezitking123 this what you get for tryna ge...,NOT,cheezitk thi get tryna get kfc popey isnt supe...
997,1130037092845670400,earphones ko 😭😭😭😭😭😭😭,NOT,earphon ko
998,1127028455651123201,RT @nj_linguist: @realgonegirl @elivalley I th...,NOT,rt njlinguist realgonegirl elivalley think peo...


In [50]:
new = pd.DataFrame({"processed":df['processed_text'],
                   "label":df['task1']}

)



In [51]:
new

,processed,label
0,hate wen femal hit ah nigga tht bro im tryna m...,HOF
1,rt airjunebug bay realli ny nigga heart w supp...,HOF
2,rt donaldjtrumpjr dear democrat american peopl...,NOT
3,rt shelovetimothi aint drug bore shit im bore,HOF
4,rt tavianjordan summer im come bore shit beach...,NOT
...,...,...
995,rt prodnos good morn everyon follow one worst ...,NOT
996,cheezitk thi get tryna get kfc popey isnt supe...,NOT
997,earphon ko,NOT
998,rt njlinguist realgonegirl elivalley think peo...,NOT


In [53]:
new.to_csv("processed_text.csv")

In [26]:
df.to_csv('processed.csv')

In [ ]:
pd.read_csv('processed.csv')
print(pd)

In [15]:
X = df['processed_text']
y = df['task1']

In [16]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [18]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_vec, y_train)

SVC(kernel='linear')

In [19]:
# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_vec, y_train)

DecisionTreeClassifier()

In [20]:
# k-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_vec, y_train)

KNeighborsClassifier()

In [21]:
# Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train_vec, y_train)

LogisticRegression()

In [22]:
# Classification report
y_pred = svm_model.predict(X_test_vec)
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         HOF       0.80      0.71      0.75        94
         NOT       0.77      0.84      0.80       106

    accuracy                           0.78       200
   macro avg       0.78      0.78      0.78       200
weighted avg       0.78      0.78      0.78       200



In [23]:

# Classification report for Decision Tree
y_pred_dt = dt_model.predict(X_test_vec)
report_dt = classification_report(y_test, y_pred_dt)
print("Decision Tree Classification Report:\n", report_dt)

Decision Tree Classification Report:
               precision    recall  f1-score   support

         HOF       0.71      0.74      0.73        94
         NOT       0.76      0.73      0.74       106

    accuracy                           0.73       200
   macro avg       0.73      0.74      0.73       200
weighted avg       0.74      0.73      0.74       200



In [24]:
# Classification report for KNN
y_pred_knn = knn_model.predict(X_test_vec)
report_knn = classification_report(y_test, y_pred_knn)
print("KNN Classification Report:\n", report_knn)

KNN Classification Report:
               precision    recall  f1-score   support

         HOF       0.61      0.74      0.67        94
         NOT       0.72      0.58      0.64       106

    accuracy                           0.66       200
   macro avg       0.66      0.66      0.65       200
weighted avg       0.67      0.66      0.65       200



In [25]:
# Classification report for Logistic Regression
y_pred_lr = lr_model.predict(X_test_vec)
report_lr = classification_report(y_test, y_pred_lr)
print("Logistic Regression Classification Report:\n", report_lr)

Logistic Regression Classification Report:
               precision    recall  f1-score   support

         HOF       0.82      0.69      0.75        94
         NOT       0.76      0.87      0.81       106

    accuracy                           0.79       200
   macro avg       0.79      0.78      0.78       200
weighted avg       0.79      0.79      0.78       200

